In [1]:
import glob
import os
from tqdm import tqdm
import cv2
import tifffile as tf
import torch


project_p = "/home/ao/Desktop/ieee/"
train_data_p = project_p + "data/Track1/train/images/"
dev_data_p = project_p + "data/dev/p1"
dev_data_pp = project_p + "data/dev/p2"

img_1 = glob.glob(os.path.join(train_data_p, "*.tif"))
img_2 = glob.glob(os.path.join(dev_data_p, "*.tif"))
img_3 = glob.glob(os.path.join(dev_data_pp, "*.tif"))

In [2]:
l = img_1 + img_2 + img_3

In [3]:
# "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-resnest/resnest269-0cc87c48.pth" to /home/ao/.cache/torch/hub/checkpoints/resnest269-0cc87c48.pth

In [4]:
ins = []
for i in range(6):
    c = {}
    c["name"] = f"channel{i+1}"
    c["data"] = []
    c["min"] = []
    c["max"] = []
    c["mean"] = []
    ins.append(c)
ins

[{'name': 'channel1', 'data': [], 'min': [], 'max': [], 'mean': []},
 {'name': 'channel2', 'data': [], 'min': [], 'max': [], 'mean': []},
 {'name': 'channel3', 'data': [], 'min': [], 'max': [], 'mean': []},
 {'name': 'channel4', 'data': [], 'min': [], 'max': [], 'mean': []},
 {'name': 'channel5', 'data': [], 'min': [], 'max': [], 'mean': []},
 {'name': 'channel6', 'data': [], 'min': [], 'max': [], 'mean': []}]

In [5]:
ins_fin = []
for i in range(6):
    c = {}
    c["name"] = f"channel{i+1}"
    c["min"] = []
    c["max"] = []
    c["mean"] = []
    ins_fin.append(c)
ins_fin

[{'name': 'channel1', 'min': [], 'max': [], 'mean': []},
 {'name': 'channel2', 'min': [], 'max': [], 'mean': []},
 {'name': 'channel3', 'min': [], 'max': [], 'mean': []},
 {'name': 'channel4', 'min': [], 'max': [], 'mean': []},
 {'name': 'channel5', 'min': [], 'max': [], 'mean': []},
 {'name': 'channel6', 'min': [], 'max': [], 'mean': []}]

In [6]:
# i = l[0]
for i in tqdm(l):
    sar = tf.imread(i)
    sar = torch.tensor(sar).permute(2, 0, 1)
    for i in range(len(sar)):
        t = sar[i]
        ins[i]["mean"].append(t.to(float).mean().item())
        ins[i]["data"].append(t)
        ins[i]["max"].append(t.to(float).max().item())
        ins[i]["min"].append(t.to(float).min().item())

100%|██████████| 2331/2331 [00:05<00:00, 440.13it/s]


In [7]:
from statistics import mean

for i in range(6):
    print(f"max_c{i+1}", max(ins[i]["max"]), end=" ")
    ins_fin[i]["max"] = max(ins[i]["max"])
print(" ")

for i in range(6):
    print(f"min_c{i+1}", min(ins[i]["min"]), end=" ")
    ins_fin[i]["min"] = min(ins[i]["min"])
print(" ")

for i in range(6):
    print(f"mean_c{i+1}", mean(ins[i]["mean"]), end=" ")
    ins_fin[i]["mean"] = mean(ins[i]["mean"])
print(" ")

for i in range(6):
    ct = torch.cat(ins[i]["data"], dim=0).to(float)
    p = ct.std().item()
    ins_fin[i]["std"] = p
    print(f"std_c{i+1}", p, end=" ")

max_c1 32765.0 max_c2 32765.0 max_c3 3343.0 max_c4 3310.0 max_c5 100.0 max_c6 255.0  
min_c1 0.0 min_c2 0.0 min_c3 -9999.0 min_c4 -33.0 min_c5 0.0 min_c6 0.0  
mean_c1 1774.172843321052 mean_c2 382.70600456948034 mean_c3 32.83576673379225 mean_c4 158.46333796412378 mean_c5 31.623006319089388 mean_c6 3.7178739697469863  
std_c1 2797.2916273636556 std_c2 710.5693234469384 std_c3 1173.4579685284687 std_c4 342.08553456322215 std_c5 17.937904473111903 std_c6 17.983585131110953 

In [9]:
import gc

del ins, ct
gc.collect()
ins_fin

[{'name': 'channel1',
  'min': 0.0,
  'max': 32765.0,
  'mean': 1774.172843321052,
  'std': 2797.2916273636556},
 {'name': 'channel2',
  'min': 0.0,
  'max': 32765.0,
  'mean': 382.70600456948034,
  'std': 710.5693234469384},
 {'name': 'channel3',
  'min': -9999.0,
  'max': 3343.0,
  'mean': 32.83576673379225,
  'std': 1173.4579685284687},
 {'name': 'channel4',
  'min': -33.0,
  'max': 3310.0,
  'mean': 158.46333796412378,
  'std': 342.08553456322215},
 {'name': 'channel5',
  'min': 0.0,
  'max': 100.0,
  'mean': 31.623006319089388,
  'std': 17.937904473111903},
 {'name': 'channel6',
  'min': 0.0,
  'max': 255.0,
  'mean': 3.7178739697469863,
  'std': 17.983585131110953}]

In [10]:
import pickle

f = open("ins.pkl", "wb")
pickle.dump(ins_fin, f)

In [17]:
ins_fin[1]

{'name': 'channel2',
 'min': 0.0,
 'max': 32765.0,
 'mean': 382.70600456948034,
 'std': 710.5693234469384}

In [3]:
s = [
    {"min": 0.0, "max": 32765.0, "mean": 1774.172843321052, "std": 2797.2916273636556},
    {"min": 0.0, "max": 32765.0, "mean": 382.70600456948034, "std": 710.5693234469384},
    {
        "min": -9999.0,
        "max": 3343.0,
        "mean": 32.83576673379225,
        "std": 1173.4579685284687,
    },
    {
        "min": -33.0,
        "max": 3310.0,
        "mean": 158.46333796412378,
        "std": 342.08553456322215,
    },
    {"min": 0.0, "max": 100.0, "mean": 31.623006319089388, "std": 17.937904473111903},
    {"min": 0.0, "max": 255.0, "mean": 3.7178739697469863, "std": 17.983585131110953},
]

In [4]:
s

[{'min': 0.0,
  'max': 32765.0,
  'mean': 1774.172843321052,
  'std': 2797.2916273636556},
 {'min': 0.0,
  'max': 32765.0,
  'mean': 382.70600456948034,
  'std': 710.5693234469384},
 {'min': -9999.0,
  'max': 3343.0,
  'mean': 32.83576673379225,
  'std': 1173.4579685284687},
 {'min': -33.0,
  'max': 3310.0,
  'mean': 158.46333796412378,
  'std': 342.08553456322215},
 {'min': 0.0,
  'max': 100.0,
  'mean': 31.623006319089388,
  'std': 17.937904473111903},
 {'min': 0.0,
  'max': 255.0,
  'mean': 3.7178739697469863,
  'std': 17.983585131110953}]

In [8]:
for i in range(len(s)):
    print(s[i]["mean"], end=", ")

1774.172843321052, 382.70600456948034, 32.83576673379225, 158.46333796412378, 31.623006319089388, 3.7178739697469863, 